# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4976, uptime 0:00:33
mini_beamline                    RUNNING   pid 4977, uptime 0:00:33
random_walk                      RUNNING   pid 4978, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4992, uptime 0:00:32
random_walk_vert                 RUNNING   pid 4980, uptime 0:00:33
simple                           RUNNING   pid 4981, uptime 0:00:33
thermo_sim                       RUNNING   pid 4982, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4983, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-18 20:55:14
Persistent Unique Scan ID: '97110a0c-38f8-40d8-8ccf-87e1a96d1947'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:55:15.1 |     -1.000 |     102242 |
|         2 | 20:55:15.2 |     -0.500 |     102631 |
|         3 | 20:55:15.2 |      0.000 |     103693 |
|         4 | 20:55:15.2 |      0.500 |     102043 |
|         5 | 20:55:15.2 |      1.000 |     100544 |
+-----------+------------+------------+------------+


generator scan ['97110a0c'] (scan num: 3)





('97110a0c-38f8-40d8-8ccf-87e1a96d1947',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/97110a0c-38f8-40d8-8ccf-87e1a96d1947-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-18 20:55:15
Persistent Unique Scan ID: '9d639f68-707e-4ec1-9efd-416c266b02cd'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:55:15.7 |          -1 | 1095421436 |


|         2 | 20:55:15.8 |          -0 | 1088236691 |


|         3 | 20:55:15.9 |           0 | 1076632595 |
|         4 | 20:55:16.0 |           0 | 1069931176 |


|         5 | 20:55:16.1 |           1 | 1065142202 |


+-----------+------------+-------------+------------+
generator scan ['9d639f68'] (scan num: 4)





('9d639f68-707e-4ec1-9efd-416c266b02cd',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-18 20:55:16
Persistent Unique Scan ID: '12d661ae-2159-49ac-a724-b31bb9e487c9'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:55:16.6 | 1053772011 |


|         2 | 20:55:17.6 | 1088691992 |


|         3 | 20:55:18.6 | 1131650974 |


...

|        28 | 20:55:43.6 | 1093630283 |


|        29 | 20:55:44.6 | 1053280168 |


|        30 | 20:55:45.6 | 1091931582 |


+-----------+------------+------------+
generator count ['12d661ae'] (scan num: 5)





('12d661ae-2159-49ac-a724-b31bb9e487c9',)